In [1]:
import numpy as np
from scipy.sparse import random, csr_matrix
import Scripts.simulate_p_circuit as simulate_p_circuit
import Scripts.CustomScripts as CS
import os
J, h, node_order = CS.load_npz_data("8bIntFac_v6.npz", target_folder="Circuit_Library")

print(h)
print(len(h))
print(J[62])

Found folder: C:\repositories\ProbabilisticComputing\Circuit_Library
[ 3.  2. -2. -3. -3. -2. -3.  3.  2.  4. -2.  4.  3. -2. -3. -3. -2.  2.
  2.  4.  3.  2.  2.  3.  4.  2.  3.  4. -2. -2.  4.  3. -2.  3.  2. -2.
 -2.  2. -1.  4.  3.  2.  3.  4. -2.  4.  3. -2.  5.  3.  2.  4.  3. -2.
  2. -2. -2.  4.  3. -2.  4.  4. -2.  3.  3.  2. -2.  4.  3. -2.  3. -2.
  4.  3.  3.  4.  4.  4.  4.  4.  4.  4.  4. -1. -2. -2. -2. -2.  2. -2.
  2.]
91
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -3.
  0.  0.  0.  0.  0.  0.  2.  2.  0. -2.  3.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]


In [ ]:
from scipy.sparse import csr_matrix
import Scripts.CustomScripts as CS
# Function to generate `.mem` files for Compressed-Row-Sparse (CSR) matrix
def generate_csr_mem_files(J_bipolar, h_bipolar, file_prefix):
    # Ensure the "bram" folder exists
    bram_folder = "bram"
    if not os.path.exists(bram_folder):
        os.makedirs(bram_folder)

    # Convert bipolar J and h to binary format
    J_binary, h_binary = CS.bipolar_to_binary(J_bipolar,h_bipolar)

    # Convert J to Compressed Sparse Row (CSR) format
    J_sparse = csr_matrix(J_binary)

    # Extract CSR components
    values = J_sparse.data
    col_indices = J_sparse.indices
    row_ptr = J_sparse.indptr
    print("Values",values)
    print("Col indices",col_indices)
    print("Row Ptr", row_ptr)

    # Construct file paths
    values_file = os.path.join(bram_folder, f"{file_prefix}_values.mem")
    col_indices_file = os.path.join(bram_folder, f"{file_prefix}_col_indices.mem")
    row_ptr_file = os.path.join(bram_folder, f"{file_prefix}_row_ptr.mem")
    h_file = os.path.join(bram_folder, f"{file_prefix}_h.mem")

    # Write the values memory file
    with open(values_file, "w") as f:
        for v in values:
            bin_value, _ = CS.decimal_to_s4_3(v)
            f.write(bin_value + "\n")

    # Write the col_indices memory file
    with open(col_indices_file, "w") as f:
        for c in col_indices:
            f.write(f"{c:08b}\n")  # Store column indices as 8-bit binary

    # Write the row_ptr memory file (Delta Encoded)
    with open(row_ptr_file, "w") as f:
        for r in row_ptr:
            f.write(f"{r:016b}\n")  # Store as 16-bit binary

    # Write the h memory file (Bias vector)
    with open(h_file, "w") as f:
        for i in range(h_binary.shape[0]):
            bin_value, _ = CS.decimal_to_s4_3(h_binary[i])
            f.write(bin_value + "\n")

    print(f"RLE-encoded mem files generated in folder '{bram_folder}':")
    print(f"  {file_prefix}_values.mem")
    # print(f"  {file_prefix}_run_lengths.mem")
    print(f"  {file_prefix}_col_indices.mem")
    print(f"  {file_prefix}_row_ptr.mem")
    print(f"  {file_prefix}_h.mem")

In [ ]:
generate_csr_mem_files(J,h,"4bIntFac")

Values [-2.  4. -4. -4. -2.  4. -4.  6. -4. -4.  4. -4.  6.  4.  4. -2.  4. -2.
 -4.  4. -4.  4. -4.  4.  4. -2. -4. -6. -4.  6.  4.  4. -4.  6.  4.  4.
  6.  6. -2.  4. -4.  6. -6.  6. -4. -2.  4. -4. -4.  6. -2.  6.  4. -2.
  4. -6.  4.  4. -4.  6. -4. -2. -4. -2.  4.  4.  6.  6. -2.  4. -4.  4.
  4.  4. -4.  4.  4. -2. -6. -4.  6. -4. -4. -4. -2. -4.  4.  4. -6. -4.
  6. -6.  6. -4. -4. -4. -6.  6. -4. -4. -4.  6.  6. -2. -4. -2.  4. -4.
 -4. -2. -4. -4. -4. -6. -4.  6. -6. -4.  6. -4. -4. -4. -4. -2. -4. -2.
  6.  6. -2. -4.  4. -6. -4. -4. -4.  6. -4. -4. -2.  4.  6.  6. -2.  4.
 -2. -4.  4.  4. -6. -4.  6. -6. -4.  6.  4.  4. -2. -4. -4. -2.  4. -4.
  6.  6. -2.  4. -6.  4.  4. -4.  6. -4. -4. -6.  6. -4. -2.  4. -4. -4.
 -6. -4.  6. -6. -4.  6.  4.  4.  4.  4. -6. -4.  6. -6.  6. -4. -4. -4.
 -2.  4.  4.  4.  6.  6. -2. -4. -2. -4. -4. -4. -2. -4. -6. -4. -4. -4.
  6. -4. -4. -2.  4.  6.  6. -2.  4. -2. -4. -6.  4.  4. -4.  6. -4. -2.
 -4. -2.  4. -4.  6.  6. -2.  4. -6.  4.  4.

In [ ]:
J_binary, h_binary = CS.bipolar_to_binary(J,h)
m = 
for i in range(len(h_binary)):  # Update each of the p-bits sequentially.
    # Compute the input to p-bit i.
    I_i = (h[i] + J[i].dot(m))

    # Determine the activation probability using a shifted and scaled tanh.
    activation_function = (np.tanh(I_i) + 1) / 2
    random_value = np.random.uniform(0, 1)
    m[i] = threshold(random_value, activation_function)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [5]:
import numpy as np

# Function to load a .mem file and convert it to Q3.4 fixed-point format
def load_q4_3_mem_file(filename):
    """
    Load a .mem file where each line is an 8-bit two's complement binary string
    in s[4][3] fixed-point format, and convert it to a numpy float array.
    """
    with open(filename, "r") as f:
        data = [CS.s4_3_to_decimal(line.strip()) for line in f]
    return np.array(data, dtype=np.float32)

# Function to decode Run-Length Encoding (RLE)
def run_length_decode(values, run_lengths):
    decoded = []
    for v, r in zip(values, run_lengths):
        decoded.extend([v] * r)
    return np.array(decoded, dtype=np.float32)  # Ensure floating-point format

# Load the `.mem` files with Q3.4 handling
values = load_q4_3_mem_file("bram/4bIntFac_values.mem")       # Nonzero J values (Q3.4 fixed-point)
col_indices = np.loadtxt("bram/4bIntFac_col_indices.mem", dtype=np.uint8, converters={0: lambda s: int(s, 2)})
row_ptr = np.loadtxt("bram/4bIntFac_row_ptr.mem", dtype=np.uint16, converters={0: lambda s: int(s, 2)})
run_lengths = np.loadtxt("bram/4bIntFac_run_lengths.mem", dtype=np.uint8, converters={0: lambda s: int(s, 2)})
# h = load_q4_3_mem_file("bram/test_h.mem")            # Bias vector h (also in Q3.4 format)

# Reconstruct sparse matrix
num_Pbits = len(h)  # The number of P-bits is determined by h
J_reconstructed = np.zeros((num_Pbits, num_Pbits), dtype=np.float32)  # Floating-point matrix

for i in range(num_Pbits):
    start_idx = row_ptr[i]
    end_idx = row_ptr[i + 1]
    
    for j in range(start_idx, end_idx):
        col = int(col_indices[j])  # Ensure index is integer
        J_reconstructed[i, col] = values[j]  # Assign Q3.4 converted value
np.set_printoptions(threshold=np.inf, linewidth=200)
# Print matrix for verification
print(values)
print("Reconstructed J matrix (floating-point Q3.4):")
print(J_reconstructed)


[-2.  4. -4. -4. -2.  4. -4.  6. -4. -4.  4. -4.  6.  4.  4. -2.  4. -2. -4.  4. -4.  4. -4.  4.  4. -2. -4. -6. -4.  6.  4.  4. -4.  6.  4.  4.  6.  6. -2.  4. -4.  6. -6.  6. -4. -2.  4. -4. -4.
  6. -2.  6.  4. -2.  4. -6.  4.  4. -4.  6. -4. -2. -4. -2.  4.  4.  6.  6. -2.  4. -4.  4.  4.  4. -4.  4.  4. -2. -6. -4.  6. -4. -4. -4. -2. -4.  4.  4. -6. -4.  6. -6.  6. -4. -4. -4. -6.  6.
 -4. -4. -4.  6.  6. -2. -4. -2.  4. -4. -4. -2. -4. -4. -4. -6. -4.  6. -6. -4.  6. -4. -4. -4. -4. -2. -4. -2.  6.  6. -2. -4.  4. -6. -4. -4. -4.  6. -4. -4. -2.  4.  6.  6. -2.  4. -2. -4.  4.
  4. -6. -4.  6. -6. -4.  6.  4.  4. -2. -4. -4. -2.  4. -4.  6.  6. -2.  4. -6.  4.  4. -4.  6. -4. -4. -6.  6. -4. -2.  4. -4. -4. -6. -4.  6. -6. -4.  6.  4.  4.  4.  4. -6. -4.  6. -6.  6. -4.
 -4. -4. -2.  4.  4.  4.  6.  6. -2. -4. -2. -4. -4. -4. -2. -4. -6. -4. -4. -4.  6. -4. -4. -2.  4.  6.  6. -2.  4. -2. -4. -6.  4.  4. -4.  6. -4. -2. -4. -2.  4. -4.  6.  6. -2.  4. -6.  4.  4.
 -4.  6. -4. -4

In [6]:
J_reconstructed[0][1]

np.float32(-2.0)